## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
df = pd.read_csv('credit_scoring_eng.csv')
print(df.head())
print(df.tail())
print(df.info())
print(df.describe())

   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

                   purpose  
0    purchase of the house 

### Conclusion

After examine the general information I realized that in order to answer the main project questions I need only 5 columns: 'children', 'family_status_id', 'total_income', 'purpose' and 'debt'.
The next step is to check this columns for missing values: 

### Step 2. Data preprocessing

### Processing missing values

In [2]:
print(df[df['children'].isnull()].count())
print(df[df['total_income'].isnull()].count())
print(df[df['family_status'].isna()].count())
print(df[df['purpose'].isna()].count())
print(df[df['debt'].isnull()].count())
print(df.loc[df['children'] == -1].count()) # this values don't make sense
print(df.loc[df['children'] == 20].count()) # this values don't make sense


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
f

There is approximately 10% of missing values in 'total_income' column (2174/21525*100 = 10.09%) 
and less than 1% of "non sense" values in children column, which is pretty low and deleting these rows wouldn't really affect the results. 
I decided to get rid of "non sense" values and replace the missing 'total_income' values with median income, which is more represent the real common income on market.

### Data type replacement

In [3]:
median = df['total_income'].median()
print(median)
df['total_income'] = df['total_income'].fillna(value = median)
# I exclude all the rows with -1, 20 values from the data:
Not = pd.Series([-1, 20])
df = df.loc[~df['children'].isin(Not)]
# checking the results:
print(df.describe()) 
print(df[df['total_income'].isnull()].count())

23202.87
           children  days_employed     dob_years  education_id  \
count  21402.000000   19240.000000  21402.000000  21402.000000   
mean       0.473180   63159.820777     43.300206      0.817120   
std        0.751837  140928.943329     12.579055      0.548741   
min        0.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.235601     33.000000      1.000000   
50%        0.000000   -1203.934202     42.000000      1.000000   
75%        1.000000    -289.740178     53.000000      1.000000   
max        5.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21402.000000  21402.000000   21402.000000  
mean           0.973133      0.080927   26429.257048  
std            1.420536      0.272729   15678.706707  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   17234.608500  
50%            0.000000      0.000000   23202.870000  
75%        

The data is ready for next step

### Processing duplicates

In [4]:
dup_befor = df.duplicated().sum() 
print('Number of duplicates before changing upper to lower cases:', dup_befor)
df.info()
# I check all the colums wich are object type for upper and lower cases and what are the column values:
print(df['education'].value_counts())
print(df['family_status'].value_counts())
print(df['gender'].value_counts())
print(df['income_type'].value_counts())
print(df['purpose'].value_counts())

# There is only one colomn detected with upper and lower cases and it's 'education'
df['education'] = df['education'].str.lower()
dup_after = df.duplicated().sum()
print('Number of duplicates after changing upper to lower cases:', dup_after)
df = df.drop_duplicates().reset_index(drop=True)
dup_after_drop = df.duplicated().sum()
print('Number of duplicates after dropping:', dup_after_drop)
print(df['purpose'].value_counts())

from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

# I crate a list of words that I wont to know the stemm of :

words = ['property', 'university', 'education', 'car']

for word in words:
    print('Source word - {}, after stemming - {}'.format(word, english_stemmer.stem(word)))
    
# I unite all the stemmed_word by topic:
    
for row in df['purpose']:
    for word in row.split(' '):
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'wed':
            df.loc[df['purpose'] == row, 'purpose'] = 'wedding'

            
for row in df['purpose']:
    for word in row.split(' '):
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'rent' or stemmed_word == 'transact' or stemmed_word == 'hous' or stemmed_word == 'real' or stemmed_word == 'properti':
            df.loc[df['purpose'] == row, 'purpose'] = 'real estate'
            
for row in df['purpose']:
    for word in row.split(' '):
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'car':
            df.loc[df['purpose'] == row, 'purpose'] = 'own a car' 

            
for row in df['purpose']:
    for word in row.split(' '):
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'educ' or stemmed_word == 'univers':
            df.loc[df['purpose'] == row, 'purpose'] = 'education'             
                       
            
dup_after_cat = df.duplicated().sum() # checking for new duplicates after categorizing
print('Number of duplicates after categorizing:', dup_after_cat)            
print(df['purpose'].value_counts())
df = df.drop_duplicates().reset_index(drop=True) # getting rid of the duplicates
final_dup_num = df.duplicated().sum() # final cheking
print('The final num of duplicates after categorising and droping:', final_dup_num) 




Number of duplicates before changing upper to lower cases: 54
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB
secondary education    13667
bachelor's degree       4698
SECONDARY EDUCATION      766
Secondary Education   

### Conclusion

There were same persons who appeared with Upper and Lower cases at the 'education' column, 
then there were also same persons who apperared at the 'purpose' column with the same purpose but registered under different topic, for example person who's purpose was to get married could appear more than once under: wedding ceremony, having a wedding or to have a wedding.
I simplified all the topics and united all persons with the same purpose under one topic,
then I got rid of all the duplicates.

### Categorizing Data

In [5]:

# In order to find relation between marital status and repaying a loan,
# I need to group all the married persons and separate them from all the other statuses:

print(df['family_status'].value_counts())

def married_or_not(status):
    if status == 'married':
        return 'yes'
    else:
        return 'No'
    
df['married'] = df['family_status'].apply(married_or_not) 

# In order to find the relation between income level and repaying a loan, 
# I need first to categorize income:

print(df['total_income'].describe())

def cat_income(income):
    if income < 17080:
        return 'low income'
    elif income > 17080 and income < 23202.87:
        return 'above low income'
    elif income == 23202.87:
        return 'median income'
    elif income > 23202.87 and income < 31518:
        return 'above median income'
    elif income > 31518:
        return 'high income'  
df['class_income'] = df['total_income'].apply(cat_income)    

print(df)    


married              11998
civil partnership     4107
unmarried             2770
divorced              1187
widow / widower        935
Name: family_status, dtype: int64
count     20997.000000
mean      26491.489117
std       15822.734584
min        3306.762000
25%       17078.015000
50%       23202.870000
75%       31517.406000
max      362496.645000
Name: total_income, dtype: float64
       children  days_employed  dob_years            education  education_id  \
0             1   -8437.673028         42    bachelor's degree             0   
1             1   -4024.803754         36  secondary education             1   
2             0   -5623.422610         33  secondary education             1   
3             3   -4124.747207         32  secondary education             1   
4             0  340266.072047         53  secondary education             1   
...         ...            ...        ...                  ...           ...   
20992         1   -4529.316663         43  secondary

### Conclusion

I did the necessary preparation of categorizing the data in order to answer project questions

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [6]:
# I have been asked about relation between having kids and repaying a loan in general, 
# so I dont need to categorize persons by the number of their children:

with_kids = df[df['children'] > 0]

with_kids_debt = with_kids[with_kids['debt'] == 1]

with_kids_debt_percent = with_kids_debt['children'].count() / len(with_kids)

print("The percentage of people who have children and didn't repaied their loans on time: {:.1%}".format(with_kids_debt_percent))

without_kids = df[df['children'] == 0]

without_kids_debt = without_kids[without_kids['debt'] == 1]

without_kids_debt_percent = without_kids_debt['children'].count() / len(without_kids)

print("The percentage of people who haven't children and didn't repaied their loans on time: {:.1%}".format(without_kids_debt_percent))



The percentage of people who have children and didn't repaied their loans on time: 9.3%
The percentage of people who haven't children and didn't repaied their loans on time: 7.7%


### Conclusion

As we can see from the table above there is lower percent of people with debt who hasn't children,
I can assume that people with children need to spend more money because of their children and have less money left to pay their loans.



- Is there a relation between marital status and repaying a loan on time?

In [7]:
debt_married_status = df.pivot_table(index = 'married', values = 'debt', aggfunc = 'sum')

debt_married_status['total_count'] = df['married'].value_counts()

debt_married_status['debt_percentage'] = (debt_married_status['debt'] / debt_married_status['total_count'] * 100).round(decimals = 3).astype(str) + '%'

print(debt_married_status.sort_values(['debt_percentage'], ascending = False))

         debt  total_count debt_percentage
married                                   
No        805         8999          8.945%
yes       925        11998           7.71%


### Conclusion

As we can see from the table above there is lower percent of people with debt who has married status,
so I can assume that married people are more responsible committed persons to pay their debts.

- Is there a relation between income level and repaying a loan on time?

In [8]:
debt_by_income = df.pivot_table(index = 'class_income', values = 'debt', aggfunc = 'sum')

debt_by_income['total_count'] = df['class_income'].value_counts()

debt_by_income['debt_percentage'] = (debt_by_income['debt'] / debt_by_income['total_count'] * 100).round(decimals = 3).astype(str) + '%'

print(debt_by_income.sort_values(['debt_percentage'], ascending = False))

                     debt  total_count debt_percentage
class_income                                          
median income         168         1758          9.556%
above median income   388         4371          8.877%
above low income      384         4369          8.789%
low income            416         5250          7.924%
high income           374         5249          7.125%


### Conclusion

As we can see from the table above there is positive relation between income level and repaying a loan only for persons who are at the high income level. The people with high income have the smallest debt percent, all other income categories are mixed up and have no relation between income level and repaying a loan. 

- How do different loan purposes affect on-time repayment of the loan?

In [9]:
loan_purpose = df.pivot_table(index = 'purpose', values = 'debt', aggfunc = 'sum')

loan_purpose['total_count'] = df['purpose'].value_counts()

loan_purpose['debt_percentage'] = (loan_purpose['debt'] / loan_purpose['total_count'] * 100).round(decimals = 3).astype(str) + '%'

print(loan_purpose.sort_values(['debt_percentage'], ascending = False))

             debt  total_count debt_percentage
purpose                                       
own a car     399         4245          9.399%
education     369         3939          9.368%
wedding       183         2295          7.974%
real estate   779        10518          7.406%


### Conclusion

As we can see from the table above people who take loan for real estate purpose take mach seriously their obligation and pay their loans on time, I can asume that most of them are grown people and have stable income unlike students that are mostly young and having more trouble to pay their debts.   

### Step 4. General conclusion

As if I was a bank manager who incharge to permite loans for his clients, this research gives me the probability of a potential borrower to repay their loan and make the decision much easier when it lands on facts. I feel that I provided useful and important information.

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.